# Questão 21.4
For Example (20.1), establish the QP problem, given by (21.7)-(21.9), for SVM learning. \\
Solve $\alpha_p$, for $p = 1, ..., 4.$

# Example (20.1)
Define the kernel $k(\mathbf{x},\mathbf{x_i}) = (1 + \mathbf{x^T}\mathbf{x_i})^2,$ where $\mathbf{x} = (x_1,x_2)^T$ and $\mathbf{x_i} = (x_{i1}, x_{i2})^T.$
The training samples $\mathbf{x_1} = (-1,-1)$ and $\mathbf{x_4} = (+1,+1)$ belong to class 0, and $\mathbf{x_2} = (-1,+1), \mathbf{x_3} = (+1,-1)$ to class 1. \\

Expanding the kernel function, we have \\
$k(\mathbf{x}, \mathbf{x_i}) = 1 + x_1^2x_{i1}^2 + 2x_1x_2x_{i1}x_{i2} + x_2^2x_{i2}^2 +2x_1x_{i1} +2x_2x_{i2} = \phi(\mathbf{x}) \cdot \phi(\mathbf{x_i}),$ \\
where \\
$\phi(\mathbf{x}) = (1,x_1^2, \sqrt{2}x_1x_2, x_2^2, \sqrt{2}x_1, \sqrt{2}x_2)^T, $ \\
$\phi(\mathbf{x_i}) =  (1, x_{i1}^2, \sqrt{2}x_{i1}x_{i2}, x_{i2}^2, \sqrt{2}x_{i1}, \sqrt{2}x_{i2})^T.$
The feature space defined by $\phi(\mathbf{x})$ is six-dimensional. To discriminate the four examples in the feature space, we define the decision boundary in $x_1x_2 = 0$. When $x_1x_2 \ge 0$, an example is categorized into class 0, and otherwise into class 1.



# Equações (21.7)-(21.9)
By applying the Lagrange multiplier method and replacing $\mathbf{x}_p^T\mathbf{x}$ by the kernel function $k(\mathbf{x}_p,\mathbf{x})$, the ultimate objective of SVM learning is to find $\alpha_p$, for $ p = 1, ..., N$, so as to minimize the dual quadratic form \\
- (21.7) $E_{SVM} = \frac{1}{2} \sum_{p=1}^N\sum_{i=1}^N y_py_ik(\mathbf{x}_p,\mathbf{x}_i)\alpha_p\alpha_i - \sum_{p=1}^N \alpha_p$ \\

subject to \\

- (21.8) $\sum_{p=1}^N y_p\alpha_p = 0 $, \\
- (21.9) $0 \le \alpha_p \le C$, for $p = 1,...,N$, \\
where $\alpha_p$ is the weight for the kernel corresponding to the $p$th example. The kernel function $k(\mathbf{x}_p,\mathbf{x}) = \phi^T(\mathbf{x}_p)\phi(\mathbf{x})$, where the form of $\phi(\cdot)$ is implicitly defined by the choice of the kernel function and does not need to be given. When $k(\cdot)$ is a linear function, that is, $k(\mathbf{x}_p,\mathbf{x}) = \mathbf{x}_p^T\mathbf{x}$, SVM reduces to linear SVM. The popular Gaussian and polynomial kernels are, respectively given by \\
$k(\mathbf{x}_i,\mathbf{x}_j) = e^{-||x_i - x_j||^2/(2\sigma^2)} $ \\
and \\
$k(\mathbf{x}_i,\mathbf{x}_j) = (\mathbf{x}_i^T\mathbf{x}_j + \theta)^m$, \\
where $m$ is a positive integer, $\sigma > 0$, and $\theta \in R$.

- (21.7) $E_{SVM} = \frac{1}{2} \sum_{p=1}^N\sum_{i=1}^N y_py_ik(\mathbf{x}_p,\mathbf{x}_i)\alpha_p\alpha_i - \sum_{p=1}^N \alpha_p$ 

- (21.8) $\sum_{p=1}^N y_p\alpha_p = 0 $

- (21.9) $0 \le \alpha_p \le C$, for $p = 1,...,N$

Para a questão temos que encontrar os valores de $α_p$ que minimizem a função $E_{SVM}$ (Definida em 21.7), atendendo às restrições (21.8) e (21.9). Tudo isso aplicado ao exemplo 20.1. Notemos que as restrições possuem C como um parâmetro livre. Trata-se de um problema de otimização. Para isso, utilizaremos a biblioteca SciPy. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors
from scipy import optimize


Implementando a função de Kernel e colocando os inputs em um vetor $x$

In [ ]:
def kernel(x1, x2):
  return (1+np.matmul(x1.T,x2))**2

In [ ]:
x = np.array([[-1,-1], [-1,1], [1,-1], [1,1]])

Podemos imprimir o valor dos kernels para as instâncias

In [ ]:
KernelMatrix = np.zeros((4,4))
for i in range(4):
  for j in range(4):
    KernelMatrix[i][j]=kernel(x[i], x[j])
print(KernelMatrix)

[[9. 1. 1. 1.]
 [1. 9. 1. 1.]
 [1. 1. 9. 1.]
 [1. 1. 1. 9.]]


Notamos que o kernel tem uma certa simetria e não depende dos valores de i e j, apenas se são iguais ou não. De forma geral, $k(x_i, x_j)$ = 9, se $i=j$ ou 1, se $i\neq j$. 

Para a utilização do algoritmo, devemos trabalhar com as classes -1 ou +1, ou seja, que $y_i\in \{-1,+1\}$ Em seguida, os samples 1 e 4, que estão na classe 0 do exemplo 20.1, são codificados para a classe -1 para o nosso problema, e os samples 2 e 3, que eram da classe 1 no exemplo 20.1, ficam na classe +1.

In [ ]:
y= np.array([-1,1,1,-1])

Agora, vamos implementar o Problema Quadrático, as restrições (21.8) e (21.9), e implementar o vetor gradiente do problema quadrático em relação a $\alpha_p$, pois isso ajuda o resolver a otimização. Inicialmente, fixamos $C=1$, mas também testamos valores maiores e o resultado do alpha dá o mesmo. Quando informamos as restrições, também informo o gradiente das restrições.

In [ ]:
n=4
# Definindo o problema quadrático
def QuadraticProblem( alpha):
  Error = 0
  for p in range(n):
    for i in range(n):
      Error+= (0.5)*y[p]*y[i]*kernel(x[i], x[p])*alpha[p]*alpha[i]
    Error -= alpha[p]
  return Error

#Derivada em relação a alpha_p do problema quadrático

def QuadraticProblemGrad(alpha):
  sum = np.zeros(n)

  for p in range(n):
    for i in range(n):
      sum[p]+= y[p]*y[i]*kernel(x[i], x[p])*alpha[i]
    sum[p] -= 1
  return sum 

#Restrição (21.8)
def constraint1(alpha):
  sum = 0
  for i in range(n):
    sum+= y[i]*alpha[i]
  return sum

#Restrição (21.9), os bounds
C=1
b = (0, C)
bounds1 =(b,b,b,b)

#Informamos a lista de restrições. Também informamos o gradiente da restrição em relação a $alpha_p$
constraints_list = ({'type': 'eq',   'fun': lambda a:  constraint1(a), 'jac': lambda a: y})



Realizando a otimização

In [ ]:
optRes = optimize.minimize(fun=lambda a: QuadraticProblem(a),
                              x0=np.ones(n), 
                              method='SLSQP', 
                              jac=lambda a: QuadraticProblemGrad(a), 
                              constraints= constraints_list, bounds = bounds1)


In [ ]:
optRes

     fun: -0.25
     jac: array([ 2.22044605e-16, -4.44089210e-16,  4.44089210e-16, -4.44089210e-16])
 message: 'Optimization terminated successfully.'
    nfev: 3
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([0.125, 0.125, 0.125, 0.125])

Imprimindo os $\alpha_p$ resultante

In [ ]:
alpha_resultante = optRes.x
print(alpha_resultante)

[0.125 0.125 0.125 0.125]


Temos então que $α_p=0.125$, para todos os $p=1,2,3,4$. Podemos variar o valor de C para ver se encontramos diferenças. Fazendo C=10 e C=100, temos:

Para C=10

In [ ]:
C=10
b = (0, C)
bounds1 =(b,b,b,b)
optRes = optimize.minimize(fun=lambda a: QuadraticProblem(a),
                              x0=np.ones(n), 
                              method='SLSQP', 
                              jac=lambda a: QuadraticProblemGrad(a), 
                              constraints= constraints_list, bounds = bounds1)
alpha_resultante = optRes.x
print(alpha_resultante)

[0.125 0.125 0.125 0.125]


$α_p = 0.125$  para todos os p.

Para C=100

In [ ]:
C=100
b = (0, C)
bounds1 =(b,b,b,b)
optRes = optimize.minimize(fun=lambda a: QuadraticProblem(a),
                              x0=np.ones(n), 
                              method='SLSQP', 
                              jac=lambda a: QuadraticProblemGrad(a), 
                              constraints= constraints_list, bounds = bounds1)
alpha_resultante = optRes.x
print(alpha_resultante)

[0.125 0.125 0.125 0.125]


$α_p = 0.125$  para todos os p. Vemos então que aumentar o valor de C não altera nossa solução. Podemos tentar diminuir C para 0.1

In [ ]:
C=0.1
b = (0, C)
bounds1 =(b,b,b,b)
optRes = optimize.minimize(fun=lambda a: QuadraticProblem(a),
                              x0=np.ones(n), 
                              method='SLSQP', 
                              jac=lambda a: QuadraticProblemGrad(a), 
                              constraints= constraints_list, bounds = bounds1)
alpha_resultante = optRes.x
print(alpha_resultante)

[0.1 0.1 0.1 0.1]


Vemos que o $α$ bate no teto do bound, 0.1, porém podemos ver abaixo que o valor da função é -0.24, enquanto que para os outros valores de C o valor é -0.25. Ou seja, abaixar o valor de C resultou em uma menor minimização da função, o que não é o desejado.

In [ ]:
optRes

     fun: -0.24
     jac: array([-0.2, -0.2, -0.2, -0.2])
 message: 'Optimization terminated successfully.'
    nfev: 1
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([0.1, 0.1, 0.1, 0.1])

Logo, nossa resposta é $α_p = 0.125$  para todos os p. 